# Class Decorators

Class decorators are classes that include functions as state.

Defining a simple class decorator.

In [11]:
class MyClassDecorator:

    def __init__(self, func):
        print("MyClassDecorator: in __init__ registing function")
        self.func = func                    # register the function with class
        self.some_state = 0

    def __call__(self, *args, **kwargs):    # override the () function operator
        print("MyClassDecorator: in __call__ to perform decorating of args and kwargs")
        self.some_state += 1                # update some state
        self.func(*args, **kwargs)

Let's define a non-decorated function and decorate the function using the class constructor...

In [12]:
def my_function():
    print("my_function")

decorated_function = MyClassDecorator(my_function)
print("type(decorated_function) =", str(type(decorated_function))[1:-1])        # class instance
decorated_function()   # call the instance -> __call__ for "()" operator

MyClassDecorator: in __init__ registing function
type(decorated_function) = class '__main__.MyClassDecorator'
MyClassDecorator: in __call__ to perform decorating of args and kwargs
my_function


Decorating `my_function` with the class decorator using `@`...

In [13]:
@MyClassDecorator                             # Effectively MyClassDecorator(my_function)
def my_function():
    print("my_function")

my_function()

MyClassDecorator: in __init__ registing function
MyClassDecorator: in __call__ to perform decorating of args and kwargs
my_function


Decorating my_function without parameters...

In [22]:
class MyParameterisedClassDecorator:

    class MyLocalClassDecorator:                                      # you can create local class within class
        def __init__(self, func, a_parameter=False):                  # could have made it an external base class
            print("MyLocalClassDecorator: in __init__")               # so get_state() could be overloaded
            self.func = func
            self.parameter = a_parameter
            self.some_state = 0

        def __call__(self, *args, **kwargs):
            print("MyLocalClassDecorator: in __call__ to perform decorating of args and kwargs")
            if self.parameter:
                self.some_state += 1                # update some state based on decorator paramater
            return self.func(*args, **kwargs)

        def get_state(self):
            return self.parameter, self.some_state

    def __init__(self, _func=None, *, a_parameter=False):
        print("MyParameterisedClassDecorator: in __init__")
        self.parameter = a_parameter
        self.func = None
        if _func is not None:
            # decorator has been created like this MyParameterisedClassDecorator(my_function)
            print("MyParameterisedClassDecorator: no parameters have been specified as _func is", _func)
            self.func = MyParameterisedClassDecorator.MyLocalClassDecorator(_func, a_parameter=self.parameter)

    def __call__(self, *args, **kwargs):
        print("MyParameterisedClassDecorator: in __call__")
        if self.func is None:     # called like this MyParameterisedClassDecorator(a_parameter=2)(my_function)
            if len(args) != 1 or not callable(args[0]):
                raise ValueError("Expecting only a single function to decorate!")
            print("MyParameterisedClassDecorator: parameters have been specified as _func is None")
            self.func = MyParameterisedClassDecorator.MyLocalClassDecorator(args[0], a_parameter=self.parameter)
            return self.func
        return self.func(*args, **kwargs)

    def get_state(self):
        # in case where parameters are used my_function will be a MyParameterisedClassDecorator object
        if self.func is None:
            raise ValueError("Undecorated function!")
        return self.func.get_state()

print("Decorating my_function")
@MyParameterisedClassDecorator
def my_function(my_function_parameter):
    print("my_function:", my_function_parameter)

print("\nCalling my_function")
my_function('Non-parameterised class decorator')
print("\ntype(my_function) =", str(type(my_function))[1:-1])
print("my_function.func.parameter =", my_function.func.parameter)
print("my_function.func.state =", my_function.func.some_state)
print("my_function.get_state() =", my_function.get_state())         # use a method to get state

Decorating my_function
MyParameterisedClassDecorator: in __init__
MyParameterisedClassDecorator: no parameters have been specified as _func is <function my_function at 0x000001DFABFC6DD0>
MyLocalClassDecorator: in __init__

Calling my_function
MyParameterisedClassDecorator: in __call__
MyLocalClassDecorator: in __call__ to perform decorating of args and kwargs
my_function: Non-parameterised class decorator

type(my_function) = class '__main__.MyParameterisedClassDecorator'
my_function.func.parameter = False
my_function.func.state = 0
my_function.get_state() = (False, 0)


Decorating `my_function` with decorator parameters...

In [23]:
print("Decorating my_function")
@MyParameterisedClassDecorator(a_parameter=True)
def my_function(my_function_parameter):
    print("my_function:", my_function_parameter)

print("\nCalling my_function")
my_function('Parameterised class')

print("\ntype(my_function) =", str(type(my_function))[1:-1])
print("my_function.parameter = ", my_function.parameter)
print("my_function.state = ", my_function.some_state)
print("my_function.get_state() =", my_function.get_state())


Decorating my_function
MyParameterisedClassDecorator: in __init__
MyParameterisedClassDecorator: in __call__
MyParameterisedClassDecorator: parameters have been specified as _func is None
MyLocalClassDecorator: in __init__

Calling my_function
MyLocalClassDecorator: in __call__ to perform decorating of args and kwargs
my_function: Parameterised class

type(my_function) = class '__main__.MyParameterisedClassDecorator.MyLocalClassDecorator'
my_function.parameter =  True
my_function.state =  1
my_function.get_state() = (True, 1)
